In [325]:
# import pandas as pd
# import json
# from datetime import datetime
# from datetime import datetime, UTC

# # Path to your JSONL file
# category = 'Digital_Music'

# # source files
# user_reviews_jsonl_file = f"Amazon Raw Data/{category}.jsonl"
# item_metadata_jsonl_file = f"Amazon Raw Data/meta_{category}.jsonl"

# # target files
# user_reviews_raw_csv = f"Amazon Raw Data/{category}_user_reviews_raw.csv"
# item_metadata_raw_csv = f"Amazon Raw Data/{category}_item_metadata_raw.csv"
# user_reviews_target_csv = f"Amazon Processed Dataset/{category}_user_reviews.csv"
# item_metadata_target_csv = f"Amazon Processed Dataset/{category}_item_metadata.csv"


# # Read JSONL file
# data = []
# with open(user_reviews_jsonl_file, 'r', encoding='utf-8') as f:
#     for line in f:
#         data.append(json.loads(line.strip()))  # Read each line as a JSON object
# user_reviews = pd.DataFrame(data)

# # Read JSONL file
# data = []
# with open(item_metadata_jsonl_file, 'r', encoding='utf-8') as f:
#     for line in f:
#         data.append(json.loads(line.strip()))  # Read each line as a JSON object
# item_metadata = pd.DataFrame(data)

# # Perform operations (modify as needed)
# # Example: Removing duplicates
# user_reviews = user_reviews.astype(str).drop_duplicates()
# user_reviews = user_reviews[['user_id', 'parent_asin', 'rating', 'title', 'text', 'timestamp', 'helpful_vote', 'verified_purchase']]
# user_reviews['title'] = user_reviews['title'].replace('N9ne', 'None')
# user_reviews['title'] = user_reviews['title'].replace('None', '')
# user_reviews['text'] = user_reviews['text'].replace('N9ne', 'None')
# user_reviews['text'] = user_reviews['text'].replace('None', '')
# user_reviews['timestamp'] = pd.to_numeric(user_reviews['timestamp'], errors='coerce')
# user_reviews['timestamp'] = pd.to_datetime(user_reviews['timestamp'], errors='coerce', unit='ms').dt.tz_localize(UTC)

# # item_metadata = item_metadata[['parent_asin', 'title', 'average_rating', 'rating_number', 'main_category', 'features', 'description', 'price', 'store']]
# item_metadata['average_rating'] = item_metadata['average_rating'].fillna(0)
# item_metadata['average_rating'] = item_metadata['average_rating'].astype(float)

# user_reviews.to_csv(user_reviews_raw_csv, index = False)
# item_metadata.to_csv(item_metadata_raw_csv, index = False)

# print(f'Number of products = {item_metadata.shape[0]}\nNumber of reviews = {user_reviews.shape[0]}')

In [326]:
import pandas as pd
import json, random
from datetime import datetime
from datetime import datetime, UTC

# Path to your JSONL file
category = 'Video_Games'

# Source files
user_reviews_jsonl_file = f"Amazon Raw Data/{category}.jsonl"
item_metadata_jsonl_file = f"Amazon Raw Data/meta_{category}.jsonl"

# Target files
user_reviews_raw_csv = f"Amazon Raw Data/{category}_user_reviews_raw.csv"
item_metadata_raw_csv = f"Amazon Raw Data/{category}_item_metadata_raw.csv"

num_samples = 30000  # Number of records to read

# Initialize reservoir
sampled_records = []
with open(item_metadata_jsonl_file, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        record = json.loads(line.strip())

        # Reservoir sampling logic
        if len(sampled_records) < num_samples:
            sampled_records.append(record)
        else:
            rand_idx = random.randint(0, i)
            if rand_idx < num_samples:
                sampled_records[rand_idx] = record

# Convert to DataFrame
item_metadata = pd.DataFrame(sampled_records)

# ✅ Step 1: Load the 30,000 sampled ASINs into a set for fast lookup
selected_asins = set(item_metadata["parent_asin"])

# ✅ Step 2: Stream through the user reviews file and extract matching records
matched_reviews = []

with open(user_reviews_jsonl_file, "r", encoding="utf-8") as f:
    for line in f:
        review = json.loads(line.strip())

        # ✅ Keep only reviews where `parent_asin` is in the selected ASINs
        if review["parent_asin"] in selected_asins:
            matched_reviews.append(review)

user_reviews = pd.DataFrame(matched_reviews)
user_reviews = user_reviews.astype(str).drop_duplicates()
user_reviews = user_reviews[['user_id', 'parent_asin', 'rating', 'title', 'text', 'timestamp', 'helpful_vote', 'verified_purchase']]
user_reviews['timestamp'] = pd.to_numeric(user_reviews['timestamp'], errors='coerce')
user_reviews['timestamp'] = pd.to_datetime(user_reviews['timestamp'], errors='coerce', unit='ms').dt.tz_localize(UTC)
user_reviews['title'] = user_reviews['title'].replace('N9ne', 'None')
user_reviews['title'] = user_reviews['title'].replace('None', '')
user_reviews['text'] = user_reviews['text'].replace('N9ne', 'None')
user_reviews['text'] = user_reviews['text'].replace('None', '')

# item_metadata = item_metadata[['parent_asin', 'title', 'average_rating', 'rating_number', 'main_category', 'features', 'description', 'price', 'store']]
item_metadata['average_rating'] = item_metadata['average_rating'].fillna(0)
item_metadata['average_rating'] = item_metadata['average_rating'].astype(float)

user_reviews.to_csv(user_reviews_raw_csv, index = False)
item_metadata.to_csv(item_metadata_raw_csv, index = False)

print(f'Number of products = {item_metadata.shape[0]}\nNumber of reviews = {user_reviews.shape[0]}')

Number of products = 30000
Number of reviews = 988594


In [327]:
item_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   main_category    27612 non-null  object 
 1   title            30000 non-null  object 
 2   average_rating   30000 non-null  float64
 3   rating_number    30000 non-null  int64  
 4   features         30000 non-null  object 
 5   description      30000 non-null  object 
 6   price            13501 non-null  object 
 7   images           30000 non-null  object 
 8   videos           30000 non-null  object 
 9   store            28942 non-null  object 
 10  categories       30000 non-null  object 
 11  details          30000 non-null  object 
 12  parent_asin      30000 non-null  object 
 13  bought_together  0 non-null      object 
 14  subtitle         76 non-null     object 
 15  author           61 non-null     object 
dtypes: float64(1), int64(1), object(14)
memory usage: 3.7+ MB


In [328]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 988594 entries, 0 to 1000537
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   user_id            988594 non-null  object             
 1   parent_asin        988594 non-null  object             
 2   rating             988594 non-null  object             
 3   title              988594 non-null  object             
 4   text               988594 non-null  object             
 5   timestamp          988594 non-null  datetime64[ns, UTC]
 6   helpful_vote       988594 non-null  object             
 7   verified_purchase  988594 non-null  object             
dtypes: datetime64[ns, UTC](1), object(7)
memory usage: 67.9+ MB


### Which reviews should we keep?

In [329]:
review_time = user_reviews[['parent_asin', 'timestamp']]
review_time = user_reviews.groupby('parent_asin', as_index=False).agg(
    no_of_reviews=('timestamp', 'count'),
    earliest_timestamp=('timestamp', 'min')
)
review_time['earliest_year_of_review'] = review_time['earliest_timestamp'].apply(lambda x: x.year)
end_timestamp = user_reviews['timestamp'].max()
review_time['review_age_mths'] = review_time['earliest_timestamp'].apply(lambda x: (end_timestamp - x).days/30)
retain_products = set(review_time[review_time['review_age_mths'] <= 3].index)
retain_products = retain_products.union(set(item_metadata[item_metadata['average_rating'] >= 4.5].index))
item_metadata = item_metadata.loc[list(retain_products)]
item_metadata = pd.merge(left = item_metadata, right = review_time[['parent_asin', 'no_of_reviews', 'review_age_mths']], on = 'parent_asin', how = 'left')

In [330]:
unique_items = pd.Series(item_metadata['parent_asin'].unique())
unique_items.name = 'parent_asin'
user_reviews = pd.merge(left = unique_items, right = user_reviews, on = 'parent_asin', how = 'inner')

print(f'Number of products = {item_metadata.shape[0]}\nNumber of reviews = {user_reviews.shape[0]}')

Number of products = 8913
Number of reviews = 433962


In [331]:
import re, emoji, inflect
import nltk
nltk.download('words')
nltk.download('stopwords')
from nltk.corpus import stopwords, words

slang_dict = {
    "gonna": "going to", "wanna": "want to", "idk": "i do not know",
    "b4": "before", "gr8": "great", "luv": "love", "thx": "thanks"
}

shortened_dict = {
    "lite": "light", "nite": "night", "coz": "because", 
    "fav": "favorite", "biz": "business", "tmrw": "tomorrow", "u": "you"
}

# stop_words = set(stopwords.words('english'))
# negations = {"not", "no", "never", "none", "nor", "n't"}  # Keep important negations
# filtered_stopwords = stop_words - negations
word_list = set(words.words()) # Load English word dictionary

def reduce_elongation(text):
    def replace_match(match):
        word = match.group(0)
        # Only reduce if it's not a real English word
        if word.lower() not in word_list:
            return re.sub(r'(.)\1{2,}', r'\1\1', word)  # Keep max 2 repetitions
        return word

    return re.sub(r'\b(\w+?)\1+\b', replace_match, text)

def replace_number(match):
    p = inflect.engine()
    num = match.group()
    try:
        # Ensure input is a valid number
        if isinstance(num, (int, float)):  
            num = int(num)  # Convert float to int to avoid issues
            return p.number_to_words(num)
        return ''  # Skip invalid values
    except:
        return ''  # Skip errors

def clean_text(text):
    text = str(text).lower()  # Lowercase
    text = emoji.demojize(text)  # Convert emojis
    text = re.sub(r'<.*?>', ' ', text)  # Remove HTML tags
    text = re.sub(r'[^a-z0-9\s]', ' ', text)  # Remove special characters
    text = " ".join([slang_dict.get(word, word) for word in text.split()])  # Replace slang
    text = " ".join([shortened_dict.get(word, word) for word in text.split()])  # Replace shortened words
    text = reduce_elongation(text)  # Remove letter elongations
    text = re.sub(r'\b\d+\b', replace_number, text)  # Replace numbers with text
    # text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\anira\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [332]:
filtered_reviews = user_reviews.copy(deep = True)

filtered_reviews['review'] = filtered_reviews['title'] + ' ' + filtered_reviews['text']
filtered_reviews.drop(['text', 'title'], axis = 1, inplace = True)
filtered_reviews['cleaned_review'] = filtered_reviews['review'].apply(clean_text)

In [333]:
filtered_reviews['review_length'] = filtered_reviews['cleaned_review'].apply(lambda x: len(x.split()))
filtered_reviews.drop(filtered_reviews[filtered_reviews['review_length'] <= 5].index, axis = 0, inplace = True)

In [334]:
len(filtered_reviews['parent_asin'].unique())

8447

#### Eliminate reviews for products that do not have reviews after 2013

In [335]:
res = filtered_reviews.groupby(['parent_asin'], as_index=False).apply(lambda x: max(x['timestamp']))
res.columns = ['parent_asin', 'latest_timestamp']
res['year'] = res['latest_timestamp'].apply(lambda x: x.year)
res.drop(res[res['year'] <= 2013].index, axis = 0, inplace = True)
filtered_reviews = pd.merge(left = filtered_reviews, right = res['parent_asin'], on = 'parent_asin', how = 'inner')

C:\Users\anira\AppData\Local\Temp\ipykernel_30220\4237903152.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  res = filtered_reviews.groupby(['parent_asin'], as_index=False).apply(lambda x: max(x['timestamp']))


In [336]:
unique_items = pd.Series(filtered_reviews['parent_asin'].unique())
unique_items.name = 'parent_asin'
filtered_items = pd.merge(left = unique_items, right = item_metadata, on = 'parent_asin', how = 'left')
filtered_items.head()

,parent_asin,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,bought_together,subtitle,author,no_of_reviews,review_age_mths
0,B085B1Y5YY,All Electronics,KH KAM HUA PSP-S110 Replacement Battery for So...,3.9,11,[PSP-S110 Replacement Battery For SONY PSP-200...,"[- Compatible Model:, SONY Lite, PSP 2th, PSP-...",10.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'OSTENT Real 1400mAh 3.6V Lithium I...,KH KAM HUA,"[Video Games, Legacy Systems, PlayStation Syst...",{'Package Dimensions': '3.23 x 2.2 x 0.71 inch...,None,NaN,NaN,5,40.200000
1,B07SZJZV88,Video Games,Nintendo Selects: The Legend of Zelda Ocarina ...,4.9,22,[Authentic Nintendo Selects: The Legend of Zel...,[],37.42,[{'thumb': 'https://m.media-amazon.com/images/...,[],Amazon Renewed,"[Video Games, Legacy Systems, Nintendo Systems...","{'Best Sellers Rank': {'Video Games': 51019, '...",None,NaN,NaN,3,46.466667
2,B00GZ5222I,Video Games,HEX Blue - Decal Style Skin fits original PS4 ...,5.0,2,"[Skin covers top, front and side of the consol...",[WraptorSkinzTM are self adhering vinyl skins ...,16.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],WraptorSkinz,"[Video Games, PlayStation 4, Accessories, Face...",{'Pricing': 'The strikethrough price is the Li...,None,NaN,NaN,2,117.266667
3,B078TRY1C5,Video Games,Monster Hunter Official Research Commission Ke...,5.0,2,[],[],None,[{'thumb': 'https://m.media-amazon.com/images/...,[],Numskull,[],{'Pricing': 'The strikethrough price is the Li...,None,NaN,NaN,1,65.666667
4,B0BKL4RH8B,None,"yuailiur 500 Pockets Game Cards Binder,card Ho...",4.6,10,[🎮🎮【Exquisite Craftsmanship & Premium Material...,[],16.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],yuailiur,"[Video Games, Nintendo Switch, Accessories, Ca...",{'Package Dimensions': '9.76 x 7.36 x 2.13 inc...,None,NaN,NaN,1,7.600000


In [337]:
print(f'Number of products = {filtered_items.shape[0]}\nNumber of reviews = {filtered_reviews.shape[0]}')
# filtered_reviews.to_csv(user_reviews_target_csv, index = False)
# filtered_items.to_csv(item_metadata_target_csv, index = False)

Number of products = 7718
Number of reviews = 383211


# Load Filtered Data

In [338]:
# import numpy as np
# import pandas as pd
# user_reviews_target_csv = f"Amazon Processed Dataset/{category}_user_reviews.csv"
# item_metadata_target_csv = f"Amazon Processed Dataset/{category}_item_metadata.csv"
# user_reviews = pd.read_csv(user_reviews_target_csv, parse_dates=['timestamp'])
# item_metadata = pd.read_csv(item_metadata_target_csv)
user_reviews = filtered_reviews
item_metadata = filtered_items

In [339]:
user_reviews['rating'] = user_reviews['rating'].astype(float).astype(int)
user_reviews['helpful_vote'] = user_reviews['helpful_vote'].astype(int).fillna(0)
user_reviews['verified_purchase'] = user_reviews['verified_purchase'].astype(bool).astype(int)

item_metadata['rating_number'] = item_metadata['rating_number'].fillna(0)
item_metadata['rating_number'] = item_metadata['rating_number'].astype(int)
item_metadata['main_category'] = item_metadata['main_category'].fillna('Unknown')
item_metadata['main_category'] = item_metadata['main_category'].replace('None', 'Unknown')
item_metadata['price'] = item_metadata['price'].replace('None', 0.0)
item_metadata['price'] = pd.to_numeric(item_metadata['price'], errors='coerce')
item_metadata['store'] = item_metadata['store'].fillna('Unknown')

#### Eliminate short term products 

In [340]:
product_exclusion_keywords = ['clearance', 'wholesale', 'discount', 'final sale', 'trial']
pattern = r'\b(' + '|'.join(product_exclusion_keywords) + r')\b'
item_metadata['short_term_product'] = item_metadata['title'].str.contains(pattern, case=False, na=False, regex=True)
item_metadata.drop(item_metadata[item_metadata['short_term_product']].index, axis = 0, inplace = True)
item_metadata.drop(['short_term_product'], axis = 1, inplace  = True)

C:\Users\anira\AppData\Local\Temp\ipykernel_30220\1590593140.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  item_metadata['short_term_product'] = item_metadata['title'].str.contains(pattern, case=False, na=False, regex=True)


In [341]:
# Convert array to text
item_metadata.drop(item_metadata[item_metadata['title'].isna()].index, axis = 0, inplace = True)
item_metadata['features'] = item_metadata['features'].apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else str(x))
item_metadata['description'] = item_metadata['description'].apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else str(x))

category_str = ' '.join(category.split('_')) if len(user_reviews.columns.intersection(['categories'])) == 0 else item_metadata['categories']
item_metadata['context'] = item_metadata['title'] + ' ' + item_metadata['features'] + ' ' + item_metadata['description'] + ' ' + category_str + ' ' + item_metadata['main_category']
item_metadata['cleaned_context'] = item_metadata['context'].apply(clean_text)
item_metadata.drop(item_metadata[item_metadata['cleaned_context'].isna()].index, axis = 0, inplace = True)

##### Remove corresponding reviews

In [342]:
unique_items = pd.Series(item_metadata['parent_asin'].unique())
unique_items.name = 'parent_asin'
user_reviews = pd.merge(left = unique_items, right = user_reviews, on = 'parent_asin', how = 'inner')

print(f'Number of products = {item_metadata.shape[0]}\nNumber of reviews = {user_reviews.shape[0]}')

Number of products = 7714
Number of reviews = 383198


In [343]:
from scipy.stats import linregress

# reg_reviews = user_reviews.copy(deep = True)

def compute_rating_slope(df):
    slopes = {}
    
    for product, group in df.groupby('parent_asin'):
        if len(group) > 1:  # Need at least 2 points to compute a trend
            group = group.sort_values(by='timestamp')
            days_since_start = (group['timestamp'] - group['timestamp'].min()).dt.total_seconds() / (24 * 3600)

            # Aggregate ratings for the same day to prevent duplicates
            daily_avg_ratings = group.groupby(days_since_start).agg({'rating': 'mean'}).reset_index()

            if daily_avg_ratings.index.nunique() == 1:
                slopes[product] = 0 
            else:
                slope, _, _, _, _ = linregress(daily_avg_ratings['timestamp'], daily_avg_ratings['rating'])
                slopes[product] = slope
        else:
            slopes[product] = 0
    
    return slopes

# Compute slopes
slopes_dict = compute_rating_slope(user_reviews)
item_metadata['rating_trend_slope'] = item_metadata['parent_asin'].map(slopes_dict)

### User Review Compilation

In [344]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np

analyzer = SentimentIntensityAnalyzer()
user_reviews['sentiment'] = user_reviews['cleaned_review'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Compute latest review date per product
user_reviews['latest_review_date'] = user_reviews.groupby('parent_asin')['timestamp'].transform('max')

# Compute relative days since each review (relative to latest review per product)
user_reviews['days_since_review'] = (user_reviews['latest_review_date'] - user_reviews['timestamp']).dt.days

# Set decay factor (λ) - adjust this value as needed
lambda_factor = 0.01  # Higher value means faster decay

# Compute time decay weights (exponential decay function)
user_reviews['time_weight'] = np.exp(-lambda_factor * user_reviews['days_since_review'])

# Normalize weights per product (so they sum to 1)
user_reviews['time_weight'] /= user_reviews.groupby('parent_asin')['time_weight'].transform('sum')

# Compute weighted sentiment per product, keeping [-1,1] range
df_product_sentiment = user_reviews.groupby('parent_asin').apply(
    lambda x: np.average(x['sentiment'], weights=x['time_weight'])
).reset_index(name='time_weighted_sentiment')
user_reviews.drop(['latest_review_date', 'days_since_review', 'time_weight'], axis = 1, inplace = True)

C:\Users\anira\AppData\Local\Temp\ipykernel_30220\1662124481.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_product_sentiment = user_reviews.groupby('parent_asin').apply(


In [345]:
# user_reviews.to_csv(user_reviews_target_csv, index = False)
# item_metadata.to_csv(item_metadata_target_csv, index = False)

In [346]:
df_reviews_grouped = user_reviews.groupby('parent_asin')['cleaned_review'].apply(lambda x: ' '.join(x)).reset_index()
from sentence_transformers import SentenceTransformer
import torch

# Load SBERT model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("all-MiniLM-L6-v2").to(device)  # Fast & good for semantic similarity

# Function to encode aggregated reviews
def encode_reviews(text):
    return model.encode(text, convert_to_numpy=True)

# Generate embeddings for all products
df_reviews_grouped['review_sbert_embedding'] = df_reviews_grouped['cleaned_review'].apply(encode_reviews)

In [347]:
reviews_compiled = pd.merge(left = df_product_sentiment, right = df_reviews_grouped.drop(['cleaned_review'], axis = 1), on = 'parent_asin', how = 'inner')
item_metadata = pd.merge(left = item_metadata, right = reviews_compiled, on = 'parent_asin', how = 'left')
item_metadata['product_sbert_embedding'] = item_metadata['cleaned_context'].apply(encode_reviews)

In [348]:
from sklearn.preprocessing import MinMaxScaler

def min_max_scale_dataframe(df, columns = []):
    columns = df.select_dtypes(include=['number']).columns
    scaler = MinMaxScaler()
    df[columns] = scaler.fit_transform(df[columns])
    
    return df, scaler  # Return scaled DataFrame and scaler object

item_metadata, scaler = min_max_scale_dataframe(item_metadata, columns=['average_rating', 'rating_number', 'no_of_reviews', 'rating_trend_slope', 'time_weighted_sentiment'])

In [349]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(max_features=500)
# tfidf_matrix = vectorizer.fit_transform(item_metadata['cleaned_context'].tolist())

# # Convert to dense format
# tfidf_dense = tfidf_matrix.toarray()

# print(tfidf_dense.shape)

In [350]:
import numpy as np

def merge_features(df): # tfidf_matrix
    num_cols = df.select_dtypes(include=['number']).columns
    numerical_features = df[num_cols].to_numpy()
    review_sbert_embeddings = np.array(df['review_sbert_embedding'].tolist())
    product_sbert_embedding = np.array(df['product_sbert_embedding'].tolist())

    # if not isinstance(tfidf_matrix, np.ndarray):
    #     raise ValueError("TF-IDF matrix must be a NumPy array.")

    final_features = np.hstack((numerical_features, product_sbert_embedding, review_sbert_embeddings))

    return final_features

In [351]:
selected_features = item_metadata[['average_rating', 'rating_number', 'no_of_reviews', 'rating_trend_slope', 'time_weighted_sentiment', 'product_sbert_embedding', 'review_sbert_embedding']]
feature_vector = merge_features(selected_features)

In [352]:
feature_vector.shape

(7714, 773)

In [353]:
print(f'{category} || Number of products = {item_metadata.shape[0]}\nNumber of reviews = {user_reviews.shape[0]}')

Video_Games || Number of products = 7714
Number of reviews = 383198


In [354]:
item_metadata.to_csv(f'Amazon Processed Dataset/{category}_item_metadata_w_features.csv', index = False)
np.save(f'Amazon Processed Dataset/{category}_feature_vector.npy', feature_vector)

In [355]:
# import faiss

# # Initialize FAISS index for L2 similarity
# feature_dim = feature_vector.shape[1]
# index = faiss.IndexFlatL2(feature_dim)

# # Normalize feature vectors before adding (for cosine similarity)
# final_features = feature_vector / np.linalg.norm(feature_vector, axis=1, keepdims=True)

# # Add product vectors to FAISS
# index.add(final_features)

# print(f"FAISS index contains {index.ntotal} items.")
# faiss.write_index(index, f"Content-based Search Index/{category}_index.faiss")

In [356]:
# def find_similar_products(product_id, index, feature_matrix, df, top_n=5):
#     """
#     Finds the top-N most similar products to a given product ID.

#     Parameters:
#     - product_id (int): The index of the product in the DataFrame.
#     - index (faiss.Index): FAISS index containing all product embeddings.
#     - feature_matrix (np.ndarray): The full product feature matrix.
#     - df (pd.DataFrame): Original DataFrame containing product info.
#     - top_n (int): Number of similar products to return.

#     Returns:
#     - List of similar product details.
#     """
#     # Query vector for the given product
#     query_vector = feature_matrix[product_id].reshape(1, -1)

#     # Search FAISS index for top-N similar items
#     distances, indices = index.search(query_vector, top_n + 1)  # +1 to exclude self

#     # Exclude the first result if it’s the query itself
#     similar_indices = indices[0][1:] if indices[0][0] == product_id else indices[0][:top_n]

#     # Fetch similar products
#     similar_products = df.iloc[similar_indices]
    
#     return similar_products


In [357]:
# # Example: Find similar products for product at index 0
# product_id = 87127
# print(f'Searching products similar to {item_metadata.loc[product_id, 'title']}...')
# top_similar_products = find_similar_products(product_id, index, final_features, item_metadata, top_n=5)

# # Print similar products
# print("Top 5 similar products:")
# top_similar_products  # Adjust columns as needed